In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import risk_kit as rk
from pricing_models import StockPricing, FuturesPricing, OptionsPricing, \
                           SwapsPricing, BondPricing, ForwardsPricing, BDTRate, \
                           CashPricing
import scipy.optimize as opt

Questions 1 and 2 should be answered by building and calibrating a 10-period Black-Derman-Toy model for the short-rate, $r_{i,j}$. You may assume that the term-structure of interest rates observed in the market place is:

Period 1 2 3 4 5 6 7 8 9 10

Spot Rate 3.0% 3.1% 3.2% 3.3% 3.4% 3.5% 3.55% 3.6% 3.65% 3.7%

As in the video modules, these interest rates assume per-period compounding so that, for example, the market-price of a zero-coupon bond that matures in period 6 is $Z_0^6$ = $100/(1+.035)^6$ = 81.35 assuming a face value of 100.

In [2]:
n = 10
market_spot_rates = np.array([3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.55, 3.6, 3.65, 3.7]) / 100
q = 0.5

### Question 1

Assume b = 0.05 is a constant for all ii in the BDT model as we assumed in the video lectures. Calibrate the $a_i$ parameters so that the model term-structure matches the market term-structure. Be sure that the final error returned is at most $10^{-8}$.

In [3]:
b = 0.05
iterations = 200
error_threshold = 1e-8

rates, error = BDTRate.calibrate(n, q, b, market_spot_rates, iterations)

print(error)
# if error < error_threshold:
#     print('The squared error is less than the allowed threshold')

1.4451216300674596e-27


/home/gt18/anaconda3/lib/python3.7/site-packages/scipy/optimize/nonlin.py:1000: RuntimeWarning: divide by zero encountered in true_divide
  d = v / vdot(df, v)


Once your model has been calibrated, compute the price of a payer swaption with notional 1 million that expires at time t = 3 with an option strike of 0. You may assume the underlying swap has a fixed rate of 3.9\% and that if the option is exercised then cash-flows take place at times t = 4, $\ldots$ , 10. (The cash-flow at time t = i is based on the short-rate that prevailed in the previous period, i.e. the payments of the underlying swap are made in arrears.)

In [4]:
N = 1000000
fixed_rate = 0.039
start_time = 4
is_long = 1

swap_pricing = SwapsPricing(n, q, fixed_rate, start_time, is_long, rates)

n_options = 3
strike = 0

option_pricing = OptionsPricing(n_options, swap_pricing, rates, q, strike)
round(option_pricing.price * N)

4102.0

### Question 2

Repeat the previous question but now assume a value of b = 0.1.

In [5]:
b = 0.1
iterations = 200
error_threshold = 1e-8

rates, error = BDTRate.calibrate(n, q, b, market_spot_rates, iterations)

if error < error_threshold:
    print('The squared error is less than the allowed threshold')

The squared error is less than the allowed threshold


In [6]:
N = 1000000
fixed_rate = 0.039
start_time = 4
is_long = 1

swap_pricing = SwapsPricing(n, q, fixed_rate, start_time, is_long, rates)

n_options = 3
strike = 0

option_pricing = OptionsPricing(n_options, swap_pricing, rates, q, strike)
round(option_pricing.price * N)

8097.0

### Question 3

Construct a n = 10-period binomial model for the short-rate, $r_{i,j}$. The lattice parameters are: $r_{0,0}$= 5%, u=1.1, d=0.9 and q = 1-q = 1/2.

Assume that the 1-step hazard rate in node (i,j) is given by $h_{ij}$ = a $b^{j-\frac{i}{2}}$ where a = 0.01 and b = 1.01. Compute the price of a zero-coupon bond with face value F = 100 and recovery R = 20%.

In [7]:
n = 10
r0 = 0.05
u = 1.1
d = 0.9
q = 0.5

rates = StockPricing(n, r0, u, d)

hazard = {
    'a': 0.01,
    'b': 1.01,
    'recovery_rate': 0.2
}
F = 100
coupon = 0.0

bond_pricing = BondPricing(n, F, q, rates, coupon, hazard)
round(bond_pricing.price, 2)

57.22

### Question 4

The true price of 5 different defaultable coupon paying bonds with non-zero recovery are specified in worksheet ${\tt Calibration}$ in the workbook ${\tt Assignment5\_cds.xlsx}$. The interest rate is r = 5% per annum. Calibrate the six month hazard rates ${\tt A6}$ to ${\tt A16}$ by minimizing the ${\tt Sum \,Error}$ ensuring that the term structure of hazard rates are non-decreasing. 

You can model the non-decreasing hazard rates by adding constraints of the form ${\tt A6} \leq {\tt A7}, \ldots, {\tt A15} \leq {\tt A16}$. Report the hazard rate at time 0 as a percentage.



$TO\ BE\ DONE\ IN\ EXCCEL$

Answer - 1.84

### Question 5

Modify the data on the ${\tt CDS \,pricing}$ worksheet in the workbook ${\tt bonds\_and\_cds.xlsx}$ to compute a par spread in basis points for a 5yr CDS with notional principal N = 10 million assuming that the expected recovery rate R = 25%, the 3-month hazard rate is a flat 1%, and the interest rate is 5% per annum.

$TO\ BE\ DONE\ IN\ EXCCEL$

Answer - 301.51